In [1]:
import datetime
from typing import List, Tuple
import pytest

In [2]:
def split_into_smaller_intervals(
        start_date: str, end_date: str, days_per_interval: int = 30
        ) -> List[Tuple[datetime.date, datetime.date]]:
    """
    Split time interval into a list of non-overlapping adjacent
    smaller intervals.
    This can be beneficial for decreasing memory consumption at
    ClickHouse.

    :param start_date:
        start date (inclusively) of initial interval in a 'YYYY-MM-DD'
        format
    :param end_date:
        end date (inclusively) of initial interval in a 'YYYY-MM-DD'
        format
    :param days_per_interval:
        maximum number of days per any of final intervals, the last
        interval can contain less days
    :return:
        list of intervals represented by their inclusive ends
    """
    start_date = datetime.datetime.strptime(start_date, "%Y-%m-%d").date()
    end_date = datetime.datetime.strptime(end_date, "%Y-%m-%d").date()
    size_in_days = (end_date - start_date).days
    size_in_intervals = size_in_days // days_per_interval + 1
    smaller_intervals = [
        (
            start_date + datetime.timedelta(days=i * days_per_interval),
            start_date + datetime.timedelta(days=(i+1) * days_per_interval - 1)
        )
        for i in range(size_in_intervals)
    ]
    last_date = min(smaller_intervals[-1][1], end_date)
    smaller_intervals[-1] = (smaller_intervals[-1][0], last_date)
    return smaller_intervals

In [3]:
@pytest.fixture(scope='function')
def intervals():
    ''' 
    fix dates for a split_into_smaller_intervals function 
    '''    
    return split_into_smaller_intervals('2017-08-29', '2018-08-29')

def test_start_date_and_end_date_are_correct():
    '''
    Testing whether both start and end dates are correct
    '''
    assert intervals()[0][0] == datetime.datetime.strptime('2017-08-29', 
                                                           "%Y-%m-%d").date(),'Expected another start date'
    assert intervals()[-1][1] == datetime.datetime.strptime('2018-08-29', 
                                                            "%Y-%m-%d").date(), 'Expected another end date'

def test_return_is_list():
    '''
    Testing whether functions return is in list format
    '''
    assert type(intervals()) == list, 'list format is expected'
    
def test_list_components_are_tuples():
    '''
    Testing whether all lists' components are tuples
    '''
    assert all(isinstance(x, tuple) for x in intervals()) == True, 'There are others types of data in list'
    
def test_tuples_components_are_datetime_date_format():
    '''
    Testing whether all components in tuples are datetime.date format
    '''
    assert all(True in (isinstance(dat, datetime.date) for dat in tup) 
               for tup in intervals()) == True, 'There are others types of data in tuples'
    
def test_first_date_of_interval_is_earlier():
    '''
    Testing whether first date of an interval is earlier than second
    '''
    assert all((tup[0] <= tup[1]) for tup in intervals()) == True

def test_previous_equal_next():
    '''
    Testing first date of a period is equal to a last date + 1 of a previous period
    '''
    last_date = [x[1] for x in intervals()[:-1]]
    first_date = [x[0] for x in intervals()[1:]]
    assert [x + datetime.timedelta(1) for x in last_date] == first_date
    

In [7]:
@pytest.mark.parametrize('start,end,dur,expected', [
    ('2018-08-23', '2018-08-29', 10, [(datetime.date(2018, 8, 23), datetime.date(2017, 8, 29))]),
    ('2018-08-23', '2018-08-29', 3, [(datetime.date(2018, 8, 23), datetime.date(2018, 8, 25)),
                                      (datetime.date(2018, 8, 26), datetime.date(2018, 8, 28)),
                                      (datetime.date(2018, 8, 29), datetime.date(2018, 8, 29))]),
])
def test_calcs_are_correct(start, end, dur, expected):
    '''
    Checking output is correct
    '''
    assert split_into_smaller_intervals(start, end, dur) == expected

In [6]:
test_start_date_and_end_date_are_correct()
test_list_components_are_tuples()
test_return_is_list()
test_tuples_components_are_datetime_date_format()
test_first_date_of_interval_is_earlier()
test_previous_equal_next()
#test_calcs_are_correct()